# Dataframe 기본 연산 
다룰 내용 :
**스키마, Row, col, select, selectExpr 등 기본적인 행과 열 연산**

- Dataframe은 Row 타입 레코드, 컬럼 으로 구성
- 스키마 : 각 컬럼명과 테이블 타입 정의
- 파티셔닝 : 클러스터에서 물리적으로 배치되는 형태
- 파티셔닝 스키마 : 파티션을 배치하는 방법을 정의

In [30]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Spark Test") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [5]:
spark

In [10]:
# Dataframe 생성
df = spark.read.format("json").load("../data/2015-summary.json")
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [9]:
spark.read.format("json").load("../data/2015-summary.json").schema

StructType([StructField('DEST_COUNTRY_NAME', StringType(), True), StructField('ORIGIN_COUNTRY_NAME', StringType(), True), StructField('count', LongType(), True)])

In [13]:
'''
- spark 데이터 타입 생성 후 적용하는 예제
- 런타임 환경에서 스키마 일치 여부 검증
'''
from pyspark.sql.types import StructField, StructType, StringType, LongType

custom_schema = StructType(
    [
        StructField('DEST_COUNTRY_NAME', StringType(), True),
        StructField('ORIGIN_COUNTRY_NAME', StringType(), True),
        StructField('count', LongType(), False, metadata={"hello" : "world"})
    ]
)

df = spark.read.format("json").schema(custom_schema).load("../data/2015-summary.json")
df.schema


StructType([StructField('DEST_COUNTRY_NAME', StringType(), True), StructField('ORIGIN_COUNTRY_NAME', StringType(), True), StructField('count', LongType(), True)])

### 5.2 컬럼과 표현식
- `col`, `column` 함수 이용
- 컬럼은 단지 표현식
- 컬럼과 컬럼의 트랜스포메이션은 파싱된 표현식과 동일한 논리적 실행 계획으로 컴파일

In [16]:
from pyspark.sql.functions import expr

expr('(((somecol + 5) * 200) - 6) < otherCol')


Column<'((((somecol + 5) * 200) - 6) < otherCol)'>

In [17]:
df.columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

### 5.3 레코드와 로우

In [15]:
df.first()

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

In [18]:
# 로우 생성
from pyspark.sql import Row
my_row = Row("Hello", None, 1, False)

### 5.4 DataFrame 조작하기
- 5.4.1 Dataframe 생성하기
- 5.4.2 select 와 selectExpr

In [19]:
df = spark.read.format("json").load("../data/2015-summary.json")
df.createOrReplaceTempView("dfTable")

In [25]:
# 5.4.1 DF 생성하기
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, LongType

my_schema = StructType([
    StructField("some", StringType(), True),
    StructField("col", StringType(), True),
    StructField("counts", StringType(), False),
])

my_row = Row("Hello", "world", 1)
myDF = spark.createDataFrame([my_row], schema=my_schema)
myDF.show()

+-----+-----+------+
| some|  col|counts|
+-----+-----+------+
|Hello|world|     1|
+-----+-----+------+



- 컬럼이나 표현식 사용 `select`
- 문자열 표현식 사용 `selectExpr`

In [22]:
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



In [27]:
from pyspark.sql.functions import col, column

# 5.4.2 select / expr : 가장 유연한 참조 방식
# expr,col, column 등 함수를 이용하여 참조 가능
df.select(
    expr("DEST_COUNTRY_NAME"),
    col("DEST_COUNTRY_NAME"),
    column("DEST_COUNTRY_NAME")
).show(2)

+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|
|    United States|    United States|    United States|
+-----------------+-----------------+-----------------+
only showing top 2 rows



In [31]:
df.selectExpr(
    "*",
    "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry"
).show(5)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
|    United States|            Ireland|  344|        false|
|            Egypt|      United States|   15|        false|
|    United States|              India|   62|        false|
+-----------------+-------------------+-----+-------------+
only showing top 5 rows



In [32]:
df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show(3)

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+

